In [3]:
using MPI
using Dates
include("funtions.jl")
using Distributions
using Statistics
using BenchmarkTools


In [4]:
function PSO_par(d,l,u,Part_N,Max_iter)
#Iniciliazamos MPI
MPI.Init()
#Communicator
comm = MPI.COMM_WORLD
#My process rank
rank = MPI.Comm_rank(comm)
#Number of processes
size = MPI.Comm_size(comm)


h = Part_N ÷ size    #h is the same for all precesses
local_act = PSO(d, l, u, h , Max_iter)
    
if rank != 0
  MPI.send(local_act, comm; dest=0, tag=0)
else
  minl = zeros(d)
  for i = 1:size-1
    mssgrcv = MPI.recv(comm; source=i, tag=0)
    minimo = min(rosenbrock(minl, d), rosenbrock(mssgrcv, d))

    # Guardamos el mínimo
    if minimo == rosenbrock(minl, d)
      global minl = minl
      global minl = mssgrcv
    end
  end
end



MPI.Barrier(comm)
#No lo finalizamos porque queremos ejecutar este programa
#varias veces, y marca error si finalizamos el MPI
return minl
end

PSO_par (generic function with 1 method)

In [5]:
PSO_par(2,[-5,-5],[10,10],100,100)

2-element Vector{Float64}:
 0.0
 0.0

In [ ]:
Curiosamente en jupyter notebook no funciona, ya que sabemos que el mínimo está en [1,1]

In [4]:
@benchmark PSO_par(2,[-5,-5],[10,10],100,100)

BenchmarkTools.Trial: 465 samples with 1 evaluation.
 Range (min … max):   9.638 ms …  13.985 ms  ┊ GC (min … max): 0.00% … 14.90%
 Time  (median):     10.548 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.770 ms ± 650.766 μs  ┊ GC (mean ± σ):  1.78% ±  4.84%

               ▄█▄▂ ▁                                           
  ▂▁▁▂▁▁▂▁▁▂▂▃▆████▇█▇▇▆▅▇▅▃▃▁▁▂▂▂▁▁▁▂▁▂▃▂▁▁▁▁▂▁▁▁▂▁▂▂▃▃▃▃▃▂▂▃ ▃
  9.64 ms         Histogram: frequency by time         12.8 ms <

 Memory estimate: 3.21 MiB, allocs estimate: 81957.

In [7]:
#De igual manera al intentar utilizar el benchmanrk en el cluster
#marcaba un error gigantesco, por lo que se obtó por un metodo manual para 
#calcular el tiempo de ejecución

#Iniciliazamos MPI
MPI.Init()
#Communicator
comm = MPI.COMM_WORLD
#My process rank
rank = MPI.Comm_rank(comm)
#Number of processes
size = MPI.Comm_size(comm)
#Empezamos a contar el tiempo
if rank == 0
  inicial = now()
end
#Definimos los valores que tendrá el PSO
d = 2
l = [-5, -5]
u = [10, 10]
Part_N = 1000
Max_iter = 1000

pn_act = Part_N ÷ size
min_act = PSO(d, l, u, pn_act, Max_iter)
if rank != 0
  MPI.send(min_act, comm; dest=0, tag=0)
else
  minl = zeros(d)
  for i = 1:size-1
    mssgrcv = MPI.recv(comm; source=i, tag=0)
    minimo = min(rosenbrock(minl, d), rosenbrock(mssgrcv, d))

    # Guardar mínimo
    if minimo == rosenbrock(minl, d)
      global minl = minl
    else
      global minl = mssgrcv
    end
  end
end

if rank == 0
  println(minl)
  #Contamos el tiempo final
  final = now()
  println(final- inicial)
end


MPI.Barrier(comm)
MPI.Finalize()


[0.0, 0.0]
1265 milliseconds
